In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import cross_val_score
import seaborn as sns
# For manupulating the data
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
# Models under the Sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
# to calculate the performances of the models 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score # score evaluation
from sklearn.model_selection import cross_val_predict # prediction
from keras.models import Sequential
from keras.layers import Dense
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold

**Dataset**

This dataset contains occurrences of hepatitis in people. 
The dataset has been downloaded from  “hepatitis_csv dataset.” https://www.openml.org/d/55, and the code was programmed in Google Colab. 

In [ ]:
data = pd.read_csv("/content/hepatitis_csv.csv")
data.head(20)

,age,sex,steroid,antivirals,fatigue,malaise,anorexia,liver_big,liver_firm,spleen_palpable,spiders,ascites,varices,bilirubin,alk_phosphate,sgot,albumin,protime,histology,class
0,30,male,False,False,False,False,False,False,False,False,False,False,False,1.0,85.0,18.0,4.0,NaN,False,live
1,50,female,False,False,True,False,False,False,False,False,False,False,False,0.9,135.0,42.0,3.5,NaN,False,live
2,78,female,True,False,True,False,False,True,False,False,False,False,False,0.7,96.0,32.0,4.0,NaN,False,live
3,31,female,NaN,True,False,False,False,True,False,False,False,False,False,0.7,46.0,52.0,4.0,80.0,False,live
4,34,female,True,False,False,False,False,True,False,False,False,False,False,1.0,NaN,200.0,4.0,NaN,False,live
5,34,female,True,False,False,False,False,True,False,False,False,False,False,0.9,95.0,28.0,4.0,75.0,False,live
6,51,female,False,False,True,False,True,True,False,True,True,False,False,NaN,NaN,NaN,NaN,NaN,False,die
7,23,female,True,False,False,False,False,True,False,False,False,False,False,1.0,NaN,NaN,NaN,NaN,False,live
8,39,female,True,False,True,False,False,True,True,False,False,False,False,0.7,NaN,48.0,4.4,NaN,False,live
9,30,female,True,False,False,False,False,True,False,False,False,False,False,1.0,NaN,120.0,3.9,NaN,False,live


First, we obtained the unique values of each feature in the dataset using “set”. The dataset has ordinal attributes, which are qualitative values. 

In [ ]:
for fe in data:
  print(fe)
  print(set(list(data[fe])))

age
{7, 20, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67, 69, 70, 72, 78}
sex
{'male', 'female'}
steroid
{False, True, nan}
antivirals
{False, True}
fatigue
{False, True, nan}
malaise
{False, True, nan}
anorexia
{False, True, nan}
liver_big
{False, True, nan}
liver_firm
{False, True, nan}
spleen_palpable
{False, True, nan}
spiders
{False, True, nan}
ascites
{False, True, nan}
varices
{False, nan, True}
bilirubin
{0.9, 1.0, 1.3, nan, 0.7, 2.2, nan, 2.0, 1.2, 0.6, 1.4, 2.3, 0.5, nan, nan, 1.5, 3.5, 2.5, 3.0, 4.6, 4.1, nan, nan, 7.6, 1.6, 1.1, 8.0, 1.7, 2.8, 0.3, 3.2, 0.8, 4.8, 1.8, 4.2, 1.9, 0.4, 2.4, 2.9, 3.9}
alk_phosphate
{nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 26.0, 30.0, nan, nan, nan, 34.0, nan, nan, nan, nan, nan, 40.0, 44.0, 45.0, 46.0, 48.0, 50.0, 52.0, 53.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 62.0, 63.0, 65.0, 67

Now, we have to prepare the dataset and get rid of unknown values. We should find the unique values of each features, then count the unknown values and found the index of each unknown value, and then compare different data preparation approaches to determine their efficiency in using different Machine Learning algorithms. 

In [ ]:
print(data.isnull().sum())

age                 0
sex                 0
steroid             1
antivirals          0
fatigue             1
malaise             1
anorexia            1
liver_big          10
liver_firm         11
spleen_palpable     5
spiders             5
ascites             5
varices             5
bilirubin           6
alk_phosphate      29
sgot                4
albumin            16
protime            67
histology           0
class               0
dtype: int64


In [ ]:
list(data['albumin']).count(' ')

After that, The LabelEncoder is used to change the qualitative values into numerical values. Finally, the dataset should only include values such as 0, 1, and 2. To train the model, we should separate the labels/classes from the dataset.

In [ ]:
df = data.copy()
# Data has ordinal attributes, The LabelEncoder has been used to make into numerical values  
Lb = LabelEncoder()
# iterating the encoding all the attributes
for features in df.columns:
    df[features] = Lb.fit_transform(df[features])
#df.head(100)

In [ ]:
Y = df['class']
X = df.loc[:,:'histology']

Now, we split the project into two parts.  
A.	Before data preparation.

B.	After data preparation. In this part we used RepeatedStratifiedKFold function to repeat Stratified K-Fold n times with different randomization in each repetition. This allows for model accuracy to be more reliable. In this step, we applied the following methods for data prepration and data processing and missing values:
1.	Dropping rows with unknown values (NaN)
2.	Dropping the features with the most unknown values (NaN)
3.	Imputation
-	Filling with the mean or median value if its a numerical value.
-	Filling with mode if its categorical value.
-	Filling the numerical value with 0 or -999, or some other number which will not occur in the data.
-	Filling the categorical value with type new for the missing values.
